In [2]:
# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import tensorflow as tf
import pandas as pd
from keras.models import load_model
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import threading
import time    
import numpy as np

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 24 13:08:46 2020

@author: Lai_Hao_Wei
"""


model = load_model('model_best.h5')  

class ProudOfYou():
    def __init__(self):
        self.root = tk.Tk()
        self.Title = "GoodWomanOfSky"
        self.root.title(self.Title)
        self.root.geometry('1300x620')
        self.root.configure(bg = '#D1E9E9')
        self.root.resizable(False, False)
        self.DetectText = tk.StringVar()
        self.DetectText.set("ProudOfYou")
        self.CatchNumber = 1
        
        self.ProcessKeep = True
        self.Thread = threading.Thread(target=self.ThreadToDo)
        self.Thread.setDaemon(True)
        
        self.BG = Image.open('question3.jpg')
        self.BG = self.BG.resize((1300, 620), Image.ANTIALIAS)
        self.BG_jpg = ImageTk.PhotoImage(self.BG)
        self.BGLabel = tk.Label(self.root, image = self.BG_jpg)
        self.BGLabel.image = self.BG_jpg
        self.BGLabel.place(x=0, y=0, relwidth=1, relheight=1)
        
        self.Zero = Image.open('zero.jpg')
        self.Zero = self.Zero.resize((640, 480), Image.ANTIALIAS)
        self.Zero_jpg = ImageTk.PhotoImage(self.Zero)
        
        self.One = Image.open('one.jpg')
        self.One = self.One.resize((640, 480), Image.ANTIALIAS)
        self.One_jpg = ImageTk.PhotoImage(self.One)
        
        self.Two = Image.open('two.jpg')
        self.Two = self.Two.resize((640, 480), Image.ANTIALIAS)
        self.Two_jpg = ImageTk.PhotoImage(self.Two)
        
        self.Three = Image.open('three.jpg')
        self.Three = self.Three.resize((640, 480), Image.ANTIALIAS)
        self.Three_jpg = ImageTk.PhotoImage(self.Three)
        
        self.Four = Image.open('four.jpg')
        self.Four = self.Four.resize((640, 480), Image.ANTIALIAS)
        self.Four_jpg = ImageTk.PhotoImage(self.Four)
        
        self.Five = Image.open('five.jpg')
        self.Five = self.Five.resize((640, 480), Image.ANTIALIAS)
        self.Five_jpg = ImageTk.PhotoImage(self.Five)
        
        self.Six = Image.open('six.jpg')
        self.Six = self.Six.resize((640, 480), Image.ANTIALIAS)
        self.Six_jpg = ImageTk.PhotoImage(self.Six)
        
        self.Seven = Image.open('seven.jpg')
        self.Seven = self.Seven.resize((640, 480), Image.ANTIALIAS)
        self.Seven_jpg = ImageTk.PhotoImage(self.Seven)
        
        self.ImageList = []
        self.ImageList.append(self.Zero_jpg)
        self.ImageList.append(self.One_jpg)
        self.ImageList.append(self.Two_jpg)
        self.ImageList.append(self.Three_jpg)
        self.ImageList.append(self.Four_jpg)
        self.ImageList.append(self.Five_jpg)
        self.ImageList.append(self.Six_jpg)
        self.ImageList.append(self.Seven_jpg)
        self.NameList = ['KURO' , '張嘉良' , '劉千娜' , '王一龍' , '蕭天雷' , '葉彩曦' , '方文鈴' , '柯淑君']
        
        self.EmptyLabel1 = tk.Label(self.root, text = '' )
        self.EmptyLabel2 = tk.Label(self.root, text = '' )
        self.EmptyLabel3 = tk.Label(self.root, text = '' )
        
        

        self.CameraButton = tk.Button(self.root, text = 'Camera', command = self.StartCamera , width = 10 , height = 1 , bd = 10 , fg = "white" , bg = '#0000C6' , relief = "raised")
        self.CameraButton.configure(font=('Arial', 12, 'bold'))
        
        self.ChooseButton = tk.Button(self.root, text = 'Choose', command = self.OpenImage , width = 10 , height = 1 , bd = 10 , fg = "white" , bg = '#FF5809' , relief = "raised")
        self.ChooseButton.configure(font=('Arial', 12, 'bold'))
        
        self.DetectButton = tk.Button(self.root, textvariable = self.DetectText , width = 20 , height = 1 , bd = 10 , fg = "yellow" , bg = 'black' , relief = "ridge")
        self.DetectButton.configure(font=('Arial', 20, 'bold'))
        
        templogo = Image.open('logo.jpg')
        templogo = templogo.resize((640, 480), Image.ANTIALIAS)
        self.templogo_jpg = ImageTk.PhotoImage(templogo)
        
        self.ChosenCanvas = tk.Canvas(self.root, width=640, height=480 , bg = '#F0FFF0')
        self.ChosenCanvas.create_image(0, 0, anchor = tk.NW, image = self.templogo_jpg)
        self.ChosenCanvas.image = self.templogo_jpg
        
        
        self.ResultCanvas = tk.Canvas(self.root, width=640, height=480 , bg = '#F0FFF0')
        self.ResultCanvas.create_image(0, 0, anchor = tk.NW, image = self.templogo_jpg)
        self.ResultCanvas.image = self.templogo_jpg
        
        
        self.EmptyLabel1.grid(row = 0 , column = 0 , columnspan = 4)
        
        self.CameraButton.grid(row = 1 , column = 0)
        self.ChooseButton.grid(row = 1 , column = 1)
        self.EmptyLabel2.grid(row = 1 , column = 2)
        self.DetectButton.grid(row = 1 , column = 3)
        
        self.EmptyLabel3.grid(row = 2 , column = 0 , columnspan = 4)
        
        self.ChosenCanvas.grid(row = 3 , column = 0 , columnspan = 2)
        self.ResultCanvas.grid(row = 3 , column = 3)
        
        self.Thread.start()
    
    def OpenImage(self):
        self.ProcessKeep = False
#         if not self.ProcessKeep:
#             self.ProcessKeep = True
#             self.TempThread = threading.Thread(target=self.ThreadToDo)
#             self.TempThread.setDaemon(True)
#             self.TempThread.start()
        
        self.ChosenCanvas.delete('all')
        self.ChosenCanvas.create_image(0, 0, anchor = tk.NW, image = self.templogo_jpg)
        self.ChosenCanvas.image = self.templogo_jpg
        
        
        FilePath = filedialog.askopenfilename(filetypes=[('jpg', '*.jpg')])
        if FilePath != '':
            img_open = Image.open(FilePath)
            img_open = img_open.resize((640, 480), Image.ANTIALIAS)
            img_choose = ImageTk.PhotoImage(img_open)
            
            self.ChosenCanvas.delete('all')
            self.ChosenCanvas.create_image(0, 0, anchor = tk.NW, image = img_choose)
            self.ChosenCanvas.image = img_choose
            ###########preprocess#####################
            
            pixels = pyplot.imread(FilePath)
            results = detector.detect_faces(pixels)
            # extract the bounding box from the first face
            x1, y1, width, height = results[0]['box']
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize((224, 224))
            image = asarray(image, 'float32')
            faces = []
            faces.append(image)
            image = asarray(faces, 'float32')
            samples = preprocess_input(image, version=2)
            yhat = model.predict(samples)
            target = np.argmax( yhat ,axis=1)
            targetnumber = target[0]
            print(target)
            print(targetnumber)
            
#             self.ProcessKeep = False
#             time.sleep(0.7)
            ThisImg = self.ImageList[targetnumber]
            ThisName = self.NameList[targetnumber]
            self.ResultCanvas.delete('all')
            self.ResultCanvas.create_image(0, 0, anchor = tk.NW, image = ThisImg)
            self.ResultCanvas.image = ThisImg
            self.DetectText.set(ThisName)
            
           ######################################################## 

        
    def StartCamera(self):
        Title = "Camera - [Q] for quit , [S] for save"
        cam = cv2.VideoCapture(0 , cv2.CAP_DSHOW)
        while True:
            ret, img = cam.read()
            # vis = img.copy()
            cv2.imshow(Title, img)
            k = cv2.waitKey(1)
            if k == ord('q'):
                break
            elif k == ord('s'):
                print("heere")
                cv2.imwrite(str(self.CatchNumber)+'.jpg',img)
                self.CatchNumber += 1
        
        cam.release()
        cv2.destroyAllWindows()
        
    
    def ThreadToDo(self):
        while self.ProcessKeep:
            for i in range(8):
                if not self.ProcessKeep:
                    break
                time.sleep(0.3)
                if not self.ProcessKeep:
                    break
                
                try:
                    if self.ProcessKeep:
                        ThisImg = self.ImageList[i]
                        ThisName = self.NameList[i]
                        # print(ThisName)
                        self.DetectText.set(ThisName)
                        self.ResultCanvas.delete('all')
                        self.ResultCanvas.create_image(0, 0, anchor = tk.NW, image = ThisImg)
                        self.ResultCanvas.image = ThisImg

                except:
                    self.ProcessKeep = False
                    
        
    def Start(self):
        self.root.mainloop()
        self.ProcessKeep = False
        
detector = MTCNN() 
Main = ProudOfYou()
Main.Start()








[1]
1
heere
heere
[3]
3
[5]
5
heere
[6]
6
